In [0]:
import sys

sys.path.append("../include_utils")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp

In [0]:
def setup_r():
    os.environ['R_HOME'] = '/home/cfriedline/g/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'],
                                                     "/home/cfriedline/lib64")

In [0]:
setup_r() #skip on mac

In [0]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext rpy2.ipython

In [0]:
def convert_GQ_to_p(q):
    return pow(10,(q/-10.0))

In [0]:
vcfutils = "perl /home/cfriedline/g/src/bcftools-1.3/vcfutils.pl"
vcftools = "/home/cfriedline/bin/vcftools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
tabix = "/home/cfriedline/gpfs/src/htslib-1.3/tabix"
bgzip = "/home/cfriedline/gpfs/src/htslib-1.3/bgzip"
java  = "/home/cfriedline/g/src/jdk1.8.0_60/bin/java"
plink = "/home/cfriedline/g/src/plink-1.07-x86_64/plink --noweb"
plink2 = "/home/cfriedline/g/src/plink_beta_3.29/plink"

# For Mac
# vcfutils = "perl /Users/chris/src/bcftools-1.3/vcfutils.pl"
# vcftools = "/Users/chris/bin/vcftools"
# bcftools = "/Users/chris/src/bcftools-1.3/bcftools"
# tabix = "/Users/chris/src/htslib-1.3/tabix"
# bgzip = "/Users/chris/src/htslib-1.3/bgzip"

In [0]:
analysis_dir = '/gpfs_fs/home/eckertlab/Mitra/mapping/split_parallel/collapsed/work/samtools1.3'
vcf_file = os.path.join(analysis_dir, "concat.vcf.gz")
assert os.path.exists(vcf_file)
vcf_file

In [0]:
rc = u.get_client(profile="sge")
dv, lv = u.get_views(rc)
len(dv)

In [0]:
def first_round_filter(args):
    vcftools, vcf_file, missing = args
    !{vcftools} --remove-indels \
    --max-missing {missing} \
    --min-alleles 2 \
    --max-alleles 2 \
    --remove-filtered-all \
    --recode \
    --recode-INFO-all \
    --gzvcf \
    {vcf_file} \
    --out {vcf_file}_{missing}

In [0]:
dv['first_round_filter'] = first_round_filter

In [0]:
jobs = []
for x in range(30, 60, 10):
    missing = x/100
    j = lv.apply_async(first_round_filter, (vcftools, vcf_file, missing))
    jobs.append(j)

In [0]:
print(jobs[0].stdout)

In [0]:
vcf_filtered = "{}_{}.recode.vcf".format(vcf_file, 0.5)
vcf_filtered_gz = "{}.gz".format(vcf_filtered)

In [0]:
vcf_filtered_gz

In [0]:
!$bgzip -c $vcf_filtered > {vcf_filtered_gz}
!$tabix {vcf_filtered_gz}

In [0]:
def get_vcf_stats(vcf_gz):
    
    stats = ['depth',
            'site-depth',
            'site-mean-depth',
            'site-quality',
            'missing-indv',
            'missing-site',
            'freq',
            'counts',
            'hardy',
            'het']
    
    for stat in stats:
        !$vcftools --gzvcf $vcf_gz \
        --out $vcf_gz \
        {"--%s" % stat} 

In [0]:
get_vcf_stats(vcf_filtered_gz)

In [0]:
pd.set_option('display.max_columns', 100)

def get_MAF(row):
    try:
        return np.min([row.A1_freq, row.A2_freq])
    except:
        print(row)
        
def get_correction(n):
    #for finite sample size
    return (2*n)/(2*n-1)

def calculate_Fis(vals):
    try:
        data = [float(x) for x in vals.split("/")]
        assert len(data) == 3
        num_individuals = np.sum(data)
        total_alleles = 2*num_individuals
        a1_count = 2*data[0]
        a2_count = 2*data[2]
        het_count = data[1]
        a1_count += het_count
        a2_count += het_count
        a1_freq = a1_count/total_alleles
        a2_freq = a2_count/total_alleles
        assert a1_freq + a2_freq == 1.0
        He = 2 * a1_freq * a2_freq * get_correction(num_individuals)
        Ho = het_count/num_individuals
        Fis = 1 - (Ho/He)
        return Fis
    except:
        return -9

def combine_vcf_stats(filedir, prefix):
    hardy_files = !ls {filedir}/{prefix}*.hwe
    hardy = pd.read_csv(hardy_files[0], sep="\t")

    hardy.columns = ['CHROM', 'POS', 'OBS(HOM1/HET/HOM2)', 'E(HOM1/HET/HOM2)', 'ChiSq_HWE',
       'P_HWE', 'P_HET_DEFICIT', 'P_HET_EXCESS']
    hardy.index = hardy.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    loci_files = !ls {filedir}/{prefix}*.l* | grep -v log
    loci_df = pd.concat([pd.read_csv(x, sep="\t", skiprows=0) for x in loci_files], axis=1)
    chrom_pos = loci_df.ix[:,0:2]
    
    frq_files = !ls {filedir}/{prefix}*.frq* | grep -v count
    frq_data = []
    h = open(frq_files[0])
    header = h.readline().strip().split()
    for line in h:
        frq_data.append(line.strip().split('\t'))

    header = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'A1_FREQ', "A2_FREQ"]
    frq_df = pd.DataFrame(frq_data)
    print(frq_df.columns)
    #frq_df = frq_df.drop([6,7],axis=1)
    frq_df.columns = header
    frq_df.index = frq_df.apply(lambda x: "%s-%s" % (x.CHROM, x.POS), axis=1)
    
    loci_df = loci_df.drop(['CHROM','CHR','POS'], axis=1)
    loci_df = pd.concat([chrom_pos, loci_df], axis=1)
    loci_df.index = loci_df.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    loci_df = pd.concat([loci_df, frq_df, hardy], axis=1)
    loci_df["A1_allele"] = loci_df.apply(lambda row: row.A1_FREQ.split(":")[0], axis=1)
    loci_df["A2_allele"] = loci_df.apply(lambda row: row.A2_FREQ.split(":")[0], axis=1)
    
    loci_df["A1_freq"] = loci_df.apply(lambda row: float(row.A1_FREQ.split(":")[1]), axis=1)
    loci_df["A2_freq"] = loci_df.apply(lambda row: float(row.A2_FREQ.split(":")[1]), axis=1)
    
    loci_df['MAF'] = loci_df.apply(get_MAF, axis=1)
    loci_df = loci_df.drop(['CHROM', 'POS'], axis=1)
    
    loci_df['Fis'] = loci_df['OBS(HOM1/HET/HOM2)'].apply(calculate_Fis)
    
    return loci_df, frq_df, hardy

In [0]:
loci_df, frq_df, hardy = combine_vcf_stats(analysis_dir, "concat.vcf.gz_0.5.recode.vcf.gz")

In [0]:
loci_df.head()

## Impute genotypes with beagle

```bash
cd /gpfs_fs/home/eckertlab/Mitra/mapping/split_parallel/collapsed/work/samtools1.3
mkdir beagle40
cd beagle40
ln -s ../concat.vcf.gz_0.5.recode.vcf.gz
java -jar ~/g/src/BEAGLE4/beagle.r1399.jar \
gl=concat.vcf.gz_0.5.recode.vcf.gz \
out=beagle40 \
nthreads=32 \
phase-its=20 \
burnin-its=20 \
impute-its=20
```

In [0]:
beagle_dir = os.path.join(analysis_dir, "beagle40")

In [0]:
beagle_dir

In [0]:
beagle_vcf_gz = os.path.join(beagle_dir, "beagle40.vcf.gz")

In [0]:
get_vcf_stats(beagle_vcf_gz)

In [0]:
loci_df_beagle, freq_df_beagle, hardy_beagle = combine_vcf_stats(beagle_dir, "beagle40")

In [0]:
loci_df_beagle.head()

In [0]:
chroms = sorted(set([x.split("-")[0] for x in loci_df.index]))

In [0]:
with open(os.path.join(analysis_dir, "chrom_map.txt"), "w") as o:
    for i, c in enumerate(chroms):
        o.write("%s\t%d\n" % (c, i))

In [0]:
def write_plink_files(vcf_gz):
    !$vcftools --gzvcf {vcf_gz} \
    --out {vcf_gz} \
    --plink \
    --chrom-map {os.path.join(analysis_dir, "chrom_map.txt")}

In [0]:
write_plink_files(vcf_filtered_gz)

In [0]:
def write_plink_recode(vcf_gz):
    !$plink --recodeA --tab --file {vcf_gz} --out {vcf_gz}_recodeA
    #!$plink --recode12 --tab --file {vcf_filtered_gz} --out {vcf_filtered_gz}_recode12

In [0]:
write_plink_recode(vcf_filtered_gz)

In [0]:
loci_df.SUM_DEPTH.describe()

In [0]:
loci_df.QUAL.plot(kind="hist");

In [0]:
loci_df.to_csv(os.path.join(analysis_dir, "loci_stats.txt"),
              sep="\t",
              index=False)

In [0]:
len(loci_df[loci_df.Fis == -9])

In [0]:
len(loci_df[loci_df.QUAL >= 10]) - len(loci_df[loci_df.QUAL >= 20])

In [0]:
len(loci_df[loci_df.QUAL < 20]), len(loci_df[loci_df.QUAL < 10])

In [0]:
len(loci_df[loci_df.Fis >= 0.5]), len(loci_df[loci_df.Fis <= -0.5]), len(loci_df[loci_df.MAF < 0.01])

In [0]:
def filter_snps(df, imputed=False):
    if imputed:
        return df[(df.MAF >= 0.01) & 
                  (df.Fis < 0.5) & 
                  (df.Fis > -0.5)]
    else:
        return df[(df.SUM_DEPTH >= 100) & 
                  (df.SUM_DEPTH < 1500) & 
                  (df.QUAL >= 20) & 
                  (df.MAF >= 0.01) & 
                  (df.Fis < 0.5) & 
                  (df.Fis > -0.5)]

In [0]:
loci_stage1 = filter_snps(loci_df)
loci_stage1.shape

In [0]:
beagle_stage1 = filter_snps(loci_df_beagle, imputed=True)
beagle_stage1.shape

In [0]:
with open(os.path.join(analysis_dir, "stage1_positions.txt"), "w") as o:
    for elem in loci_stage1.index:
        o.write("%s\n" % "\t".join(elem.split("-")))
        
with open(os.path.join(beagle_dir, "stage1_positions.txt"), "w") as o:
    for elem in beagle_stage1.index:
        o.write("%s\n" % "\t".join(elem.split("-")))
    

In [0]:
for d, vcf_gz in zip([analysis_dir, beagle_dir], [vcf_filtered_gz, beagle_vcf_gz]):
    !$vcftools --gzvcf $vcf_gz \
    --remove-indels  \
    --remove-filtered-all \
    --recode \
    --recode-INFO-all \
    --positions {os.path.join(d, "stage1_positions.txt")} \
    --out {os.path.join(d, "good_snps")}

In [0]:
for d in [analysis_dir, beagle_dir]:
    snps = os.path.join(d, "good_snps.recode.vcf")
    snps_gz = snps + ".gz"
    !$bgzip -c {snps} > {snps_gz}
    !$tabix {snps_gz}

In [0]:
def get_intersection(imp, ni):
    return set.intersection(set(ni.index), set(imp.index))

In [0]:
isect = get_intersection(beagle_stage1, loci_stage1)

In [0]:
isect = sorted(isect)

In [0]:
len(loci_stage1.index), len(beagle_stage1.index), len(isect)

In [0]:
for d in [analysis_dir, beagle_dir]:
    with open(os.path.join(d, "isect_positions.txt"), "w") as o:
        for elem in isect:
            o.write("%s\n" % "\t".join(elem.split("-")))


In [0]:
for d, vcf_gz in zip([analysis_dir, beagle_dir], [vcf_filtered_gz, beagle_vcf_gz]):
    !$vcftools --gzvcf $vcf_gz \
    --remove-indels  \
    --remove-filtered-all \
    --recode \
    --recode-INFO-all \
    --positions {os.path.join(d, "isect_positions.txt")} \
    --out {os.path.join(d, "isect_snps")}

In [0]:
for d in [analysis_dir, beagle_dir]:
    snps = os.path.join(d, "isect_snps.recode.vcf")
    snps_gz = snps + ".gz"
    !$bgzip -c {snps} > {snps_gz}
    !$tabix {snps_gz}
    
    srted = snps_gz + "_sorted.vcf"
    srted_gz = srted + ".gz"
    !vcf-sort {snps_gz} > {srted}
    !$bgzip -c {srted} > {srted_gz}
    !$tabix {srted_gz}

In [0]:
for d in [analysis_dir, beagle_dir]:
    f = os.path.join(d, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
    assert os.path.exists(f)
    write_plink_files(f)
    write_plink_recode(f)

In [0]:
for d in [analysis_dir, beagle_dir]:
    f = os.path.join(d, "isect_snps.recode.vcf.gz_sorted.vcf.gz")
    !$vcftools --gzvcf {f} \
    --out {f} \
    --012

In [0]:
!grep -c ">" /home/cfriedline/eckertlab/SugarPine_genome/pila.v1.0.scafSeq_mapped.fasta

## export file with alleles
```
zcat isect_snps.recode.vcf.gz_sorted.vcf.gz | ~/g/src/vcftools-0.1.14/src/perl/vcf-to-tab > isect_snps.recode.vcf.gz_sorted.vcf.gz.tab
```

In [0]:
raw = pd.read_csv("/gpfs_fs/home/eckertlab/Mitra/mapping/split_parallel/collapsed/work/samtools1.3/beagle40/isect_snps.recode.vcf.gz_sorted.vcf.gz_recodeA.raw",
                 sep="\t", header=0, index_col=0)

In [0]:
raw.head()